In [198]:
import pandas as pd
train=pd.read_csv('../input/song-popularity-prediction/train.csv')

In [199]:
train.head()

In [200]:
train.info()

In [201]:
#applying mean to replace null in duration of the song
import numpy as np
dur_mean=np.mean(train['song_duration_ms'])
#train['song_duration_ms'].fillna(dur_mean,inplace=True)

In [202]:
import seaborn as sns
import matplotlib.pyplot as plt
train.hist(figsize=(12,10))
plt.show()

EDA of Each Feature

In [203]:
# song_duration_ms plot
sns.set_theme()
ax=sns.displot(data=train,x='song_duration_ms',kind='kde',color='red',height=4,aspect=3) 
ax=sns.displot(data=train,x='song_duration_ms',kind='kde',height=4,aspect=3,hue='song_popularity') #Kernel Distribution Estimation

Hence from the above data we see that the songs in the range of 3mins are most popular

In [204]:
# acousticness plot
import numpy as np
ax=sns.displot(data=train,x='acousticness',kind='kde',color='orange',height=5,aspect=3)
ax=sns.displot(data=train,x=np.log(train['acousticness']),kind='kde',color='orange',height=5,aspect=3)
ax=sns.displot(data=train,x='acousticness',kind='kde',height=5,aspect=3,hue='song_popularity') 

Taking a log doesnt help much in improving the trend, so as of now we dont do a log transformation for this column

In [205]:
# danceability plot
ax=sns.displot(data=train,x='danceability',kind='kde',color='green',height=4,aspect=3)
ax=sns.displot(data=train,x='danceability',kind='kde',height=4,aspect=3,hue='song_popularity')

In [206]:
# energy plot
ax=sns.displot(data=train,x='energy',kind='kde',color='purple',height=4,aspect=3) 
ax=sns.displot(data=train,x='energy',kind='kde',height=4,aspect=3,hue='song_popularity') 

In [207]:
# instrumentalness plot

ax=sns.displot(data=train,x='instrumentalness',kind='kde',color='brown',height=4,aspect=3) 
ax=sns.displot(data=train,x=np.log(train['instrumentalness']),kind='kde',color='brown',height=4,aspect=3) 
ax=sns.displot(data=train,x='instrumentalness',kind='kde',height=4,aspect=3,hue='song_popularity') 

Using the log of the feature looks a reasonable outcome

In [208]:
# key plot
sns.displot(data=train,x='key',color='darkblue',height=4,aspect=3) 
ax=sns.displot(data=train,x='key',height=4,aspect=3,hue='song_popularity',multiple="stack") 

In [209]:
# liveness plot
ax=sns.displot(data=train,x='liveness',kind='kde',color='Magenta',height=4,aspect=3) 
ax=sns.displot(data=train,x=np.log(train['liveness']),kind='kde',color='Magenta',height=4,aspect=3)
ax=sns.displot(data=train,x='liveness',kind='kde',height=4,aspect=4,hue='song_popularity') 

Using the log of this feature as we get a good distribution using the latter

In [210]:
# loudness plot
ax=sns.displot(data=train,x='loudness',kind='kde',color='Crimson',height=4,aspect=3) 
ax=sns.displot(data=train,x='loudness',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [211]:
#audio mode
sns.displot(data=train,x='audio_mode',color='cyan',height=4,aspect=3) 
ax=sns.displot(data=train,x='audio_mode',height=4,aspect=3,hue='song_popularity',multiple="stack") 

In [212]:
#Speechiness 
ax=sns.displot(data=train,x='speechiness',kind='kde',color='lightGreen',height=4,aspect=3)
ax=sns.displot(data=train,x=np.log(train['speechiness']),kind='kde',color='lightGreen',height=4,aspect=3)
ax=sns.displot(data=train,x='speechiness',kind='kde',height=4,aspect=4,hue='song_popularity') 

Using log of this feature

In [213]:
#tempo
ax=sns.displot(data=train,x='tempo',kind='kde',color='Black',height=4,aspect=3) 
ax=sns.displot(data=train,x='tempo',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [214]:
#Time signature
sns.displot(data=train,x='time_signature',color='DarkOrange',height=4,aspect=3) 
ax=sns.displot(data=train,x='time_signature',height=4,aspect=3,hue='song_popularity',multiple="stack")

In [215]:
#Audio Valence
ax=sns.displot(data=train,x='audio_valence',kind='kde',color='tan',height=4,aspect=3) 
ax=sns.displot(data=train,x='audio_valence',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [216]:
#Song popularity
sns.displot(data=train,x='song_popularity',color='maroon',height=4,aspect=3) 

There is a class imbalance, though not hugely imbalanced. But there is some imbalance between the classes

In [217]:
train.info()

In [218]:
#converting the songs in mins
train['song_duration_ms']=train['song_duration_ms']/(1000*60)

In [219]:
train.rename(columns={'song_duration_ms':'song_duration_mins'})

In [220]:
#Tranforming columns intrumentalness, liveness, speechiness
train['log_intrumentalness']=train['instrumentalness'].apply(lambda x: np.log(x))
train['log_liveness']=train['liveness'].apply(lambda x: np.log(x))
train['log_speechiness']=train['speechiness'].apply(lambda x: np.log(x))

In [221]:
train.head()

In [256]:
#Checking correlation of the columns - with the null values
fig, ax = plt.subplots(figsize=(16,12))
ax=sns.heatmap(train.corr(),annot=True,linewidths=.5)

In [223]:
train.describe()

In [225]:
train_ins=train[train['instrumentalness'].isnull()==False]
train['instrumentalness'].fillna(np.median(train_ins['instrumentalness']),inplace=True)
train_ins=train[train['liveness'].isnull()==False]
train['liveness'].fillna(np.median(train_ins['liveness']),inplace=True)

In [226]:
train['acousticness'].fillna(np.mean(train['acousticness']),inplace=True)
train['danceability'].fillna(np.mean(train['danceability']),inplace=True)
train['energy'].fillna(np.mean(train['energy']),inplace=True)
train['key'].fillna(np.mean(train['key']),inplace=True)
train['loudness'].fillna(np.mean(train['loudness']),inplace=True)

In [227]:
train.info()

In [228]:
fig, ax = plt.subplots(figsize=(13,9))
ax=sns.heatmap(train.corr(),annot=True,linewidths=.5)

In [229]:
#Using columns acousticness, energy,loudness,audio_valence
X_train=train[['acousticness','energy','loudness','audio_valence']]
y_train=train['song_popularity']

In [230]:
#Creating a validation set
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [231]:
X_train

In [232]:
y_train

In [233]:
from sklearn.ensemble import RandomForestClassifier

ens1=RandomForestClassifier(n_estimators=100,random_state=42)
ens1.fit(X_train,y_train)

In [234]:
y_val_pred=ens1.predict(X_val)

In [235]:
#accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_val_pred)

In [236]:
test=pd.read_csv('../input/song-popularity-prediction/test.csv')

In [237]:
test.info()

In [238]:
test['song_duration_ms']=train['song_duration_ms']/1000*60

In [239]:
test.rename(columns={'song_duration_ms':'song_duration_mins'})

In [240]:
test['acousticness'].fillna(np.mean(test['acousticness']),inplace=True)
test['danceability'].fillna(np.mean(test['danceability']),inplace=True)
test['energy'].fillna(np.mean(test['energy']),inplace=True)
test['key'].fillna(np.mean(test['key']),inplace=True)
test['loudness'].fillna(np.mean(test['loudness']),inplace=True)

In [241]:
train_ins=test[test['instrumentalness'].isnull()==False]
test['instrumentalness'].fillna(np.median(train_ins['instrumentalness']),inplace=True)
train_ins=test[test['liveness'].isnull()==False]
test['liveness'].fillna(np.median(train_ins['liveness']),inplace=True)

In [242]:
test.info()

In [243]:
X_test=test[['acousticness','energy','loudness','audio_valence']]

In [244]:
sub=test[['id']]

In [245]:
sub['song_popularity']=ens1.predict(X_test)

In [246]:
sub['song_popularity'].value_counts()

In [247]:
sub.to_csv('submission.csv')